In [8]:
import pandas as pd
import matplotlib.pyplot as plt
# Display figures inline in Jupyter notebook
import seaborn as sns
import numpy as np
from pathlib import Path
from scipy import signal
import datetime
import numpy as np

In [9]:
import os,glob
g = os.getcwd()
g
path = 'D:\\repos\\CIAT\\IDB_PROJECT'
os.chdir(path)

In [10]:
df = pd.read_csv("Data_IDB/alldf.csv")
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df.head()
df.ID = df.ID.astype(int)
df.head()
type(df.date[0])

pandas._libs.tslibs.timestamps.Timestamp

# mean moth by ID 

In [4]:
ID = df[df.ID==df.ID[0]]

months = []
for year in range(2011,2020):
    for moth in range(1,13):
        Date = str(year)+"-"+str(moth)
        date_time_obj = datetime.datetime.strptime(Date, '%Y-%m')
        months.append(date_time_obj)
        
ts_id = pd.DataFrame()
for i in range(1,len(months)):
    date = ID[(ID.date>=months[i-1]) & (ID.date<=months[i])]
    if len(date)>1:
        mean = date.mean(0)
        mean["date"] = months[i-1]
    else:
        mean = date
    mean = mean[["ID","date","NDVI","OSAVI","EVI","NWVI","patca"]]
#     ts_id = pd.concat([ts_id,mean],axis=1,ignore_index=True)
# ts_id




,ID,date,NDVI,OSAVI,EVI,NWVI,patca
0,27611,2011-01-01,0.632409,0.445440,0.434433,-0.028498,0
1,27611,2011-01-17,0.599304,0.418618,0.404649,0.014193,0


In [21]:
i

108

# Smoothing data

In [11]:
len(df["ID"].unique())
smooth_all = pd.DataFrame()
for fid in df["ID"].unique():
    db = df[df.ID == fid]
    smootdf = db[["NDVI","OSAVI","EVI","NWVI"]]
    smootdf = smootdf.apply(lambda x: signal.savgol_filter(x,9, 2, mode='nearest'),0,result_type='broadcast')
    smooth_merge_ID = pd.concat([db[["ID","date"]].reset_index(drop=True), smootdf.reset_index(drop=True)], axis=1)
    smooth_all = smooth_all.append(smooth_merge_ID.reset_index(drop=True))
    
# smooth_all.to_csv("Data_IDB/alldf_smooth.csv",index=False)

In [13]:
smooth_all.date[0]
# melt_smooth = smooth_all.melt(["date","ID"])
# melt_smooth.head()
# # melt_smooth["ID"].unique()

0   2011-01-01
0   2011-02-11
0   2011-03-15
0   2011-03-31
0   2011-03-15
       ...    
0   2011-01-01
0   2011-01-01
0   2011-01-01
0   2011-01-01
0   2011-01-01
Name: date, Length: 377, dtype: datetime64[ns]

set param to plot trends

In [ ]:
# sns.set_style("white")
rc={"font.weight":"bold",'figure.figsize':(30, 6),'axes.labelsize': 28, 
    'font.size': 28,'axes.titlesize': 28,'legend.fontsize': 18, 'axes.titlesize': 28,
   "lines.linewidth": 2.5,'font.sans-serif': "Times New Roman"}
# sns.set()
sns.set(style="white", rc=rc,font_scale = 1.8)

DATE = pd.DataFrame({
    'inside':['2011-01-01','2012-01-01','2013-01-01','2014-01-01','2015-01-01','2016-01-01','2017-01-01','2018-01-01','2019-01-01']
})

DATE['inside'] = pd.to_datetime(DATE['inside'], format='%Y-%m-%d')
# DATE
# sns.set_style("white")
# sns.set_context("paper",rc=rc,font_scale = 1.5)

# test graphs plot smooth trends 

In [ ]:
for fid in melt_smooth.ID.unique():
    db_ID = melt_smooth[melt_smooth.ID == fid]# select data by ID
    db_VI = db_ID[db_ID.variable != "NWVI"] # delete the data from water index

    p = sns.lineplot('date', 'value', hue='variable',data=db_VI)
    p.set(xlabel='Dates', ylabel='VIs value',title="Smooth Time serie for ID: " + str(fid)) #+  str(df1.date.iloc[0][0:4])
    # p.set_xticklabels(p.get_xticklabels(),rotation=30, horizontalalignment='right')

    handles, labels = p.get_legend_handles_labels()
    lgd = dict(zip(labels, handles))
    p.legend(lgd.values(), lgd.keys(),loc='center left', bbox_to_anchor=(0.98, 0.5), ncol=1)
    sns.despine()
    for inside  in DATE.inside:
        p.axvline(inside,color='r',label="start year")
    # # p.xticks(rotation=45)
    # p.tick_params(axis='x', labelrotation=90)
#     sns.despine()
    figure = p.get_figure()
    figure.savefig('Data_IDB/trends/TS_all-' + str(fid)+ '.png', dpi=600)
    plt.clf()